In [2]:
import numpy as np
from define import qcode as qc
from define import chanreps as chrep
from define import chandefs as chdef
from define import globalvars as gv
from define.QECCLfid import uncorrectable as uc

In [93]:
qcode = qc.QuantumErrorCorrectingCode("5qc")
qc.Load(qcode)

In [138]:
single_probs = np.tile(np.real(np.diag(C)),[qcode.N,1])
pauliprobs = np.array([np.prod([single_probs[q,qcode.PauliOperatorsLST[p,q]] for q in range(qcode.N)]) for p in range(qcode.PauliOperatorsLST.shape[0])], dtype = np.double)
print("Single qubit probabilities\n{}\nSum\n{}".format(single_probs,np.sum(single_probs, axis=1)))
pu = uc.ComputeUnCorrProbUsingMinWt(single_probs, qcode)
print("p_u = {}".format(pu))

Single qubit probabilities
[[9.33759249e-01 4.87352675e-04 8.82691568e-03 5.69264830e-02]
 [9.33759249e-01 4.87352675e-04 8.82691568e-03 5.69264830e-02]
 [9.33759249e-01 4.87352675e-04 8.82691568e-03 5.69264830e-02]
 [9.33759249e-01 4.87352675e-04 8.82691568e-03 5.69264830e-02]
 [9.33759249e-01 4.87352675e-04 8.82691568e-03 5.69264830e-02]]
Sum
[1. 1. 1. 1. 1.]
p_u = 0.038028695253963685


In [2]:
def ProbExtendedPE(pE, pI, bi, nB):
    # Compute the probability of the padded error.
    # This is the expression in P_1 in (1).
    # P_1(E) = (pI)^(n*(i-1)) . pE . (pI)^(n*(b1 - i - 1))
    # where
    # pI = probability of the identity error on a single qubit: pI
    # n = number of physical qubits in a code block: qcode.N
    # pE = probability of the error in the code block: pE
    # i = index of the block: bi
    # b1 = number of blocks to extend over: nB
    extprob = np.power(pI, bi - 1) * pE * np.power(pI, nB - bi - 1)
    return extprob

In [3]:
def ProbExtendedIE(pE_qr, pI, bi, bj, nB):
    # Compute the probability of the interleaved error.
    # This is the expression in P_2 in (2).
    # Assume j > i.
    # P_2(E) = (pI)^(n*(i-1)) . pE_q . (pI)^(n*(j-i)) . pE_r . (pI)^(n*(b1-j-1)).
    extprob = np.power(pI, bi - 1) * pE_qr * np.power(pI, bj - bi) * np.power(pI, nB - bj - 1)
    return extprob

In [4]:
def ComputePCorrL2(errprobs, single_qubit_probs, n_blocks):
    # Compute the total probability of correctable errors for a level 2 concatenated code.
    # This is expression (3) in the notes.
    # errors["sameblock"][i] = probability of the correctable n-qubit error Ei.
    # errors["acrossblocks"][i, j] = product of probabilities of two n-qubit errors Ei and Ej such that Ei Ej is a correctable error.
    pI = errprobs["sameblock"][0]
    pcorr = 0
    for e in range(ncorr_errors):
        for i in range(n_blocks):
            pcorr += ProbExtendedPQE(errprobs["sameblock"][e], pI, i, n_blocks)
    for e in range(ncorr_errors_wt2):
        for i in range(n_blocks):
            for j in range(i + 1, n_blocks):
                pcorr += ProbExtendedIQE(errprobs["acrossblocks"][i][j], pI, i, j, n_blocks)
    return pcorr

In [ ]:
n=3
errprobs = {"sameblock": np.random.rand(4**n), "acrossblocks": np.random.rand(4**n, 4**n)}